In [1]:
import pandas as pd
with open('market_dominance_data.json', 'r') as file:
    market_dominance_data = pd.read_json(file)
    
btc_data = market_dominance_data[market_dominance_data['name'] == 'BTC']['data'].iloc[0]
btc_df = pd.DataFrame(btc_data, columns=['timestamp', 'dominance'])

# Convert timestamp to date-only human-readable form
btc_df['timestamp'] = pd.to_datetime(btc_df['timestamp'], unit='ms').dt.strftime('%Y-%m-%d')

import json
with open('BTC_price.json', 'r') as json_file:
    dataset = json.load(json_file)
# Extract the relevant information from the dataset
coin_data = dataset['coins']['coingecko:bitcoin']
timestamps = [entry['timestamp'] for entry in coin_data['prices']]
prices = [entry['price'] for entry in coin_data['prices']]

# Create a DataFrame
df = pd.DataFrame({'timestamp': timestamps, 'price': prices})

# Convert the timestamp to a datetime object
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s').dt.date

# Convert timestamp in btc_df to datetime
btc_df['timestamp'] = pd.to_datetime(btc_df['timestamp'])

# Convert timestamp in df to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Merge the two DataFrames on the 'timestamp' column
merged_df = pd.merge(btc_df, df, on='timestamp', how='inner')


In [2]:
# Load the Bitcoin 90 days data from the JSON file
with open('bitcoin_90_days.json', 'r') as json_file:
    bitcoin_90_days_data = json.load(json_file)

# Extract relevant information from the dataset
timestamps_volume = [entry[0] for entry in bitcoin_90_days_data['stats']]
txn_volumes = [entry[1] for entry in bitcoin_90_days_data['stats']]

# Create a DataFrame
volume_df = pd.DataFrame({'timestamp': timestamps_volume, 'txn_volume': txn_volumes})

# Convert the timestamp to a datetime object
volume_df['timestamp'] = pd.to_datetime(volume_df['timestamp'], unit='ms').dt.date

# Group by timestamp and take the first txn_volume for each date
volume_df = volume_df.groupby('timestamp', as_index=False).first()

# Drop the first row
volume_df = volume_df.iloc[1:]

In [3]:
volume_df

,timestamp,txn_volume
1,2023-09-08,5.098377e+11
2,2023-09-09,5.045331e+11
3,2023-09-10,5.043739e+11
4,2023-09-11,5.034897e+11
5,2023-09-12,4.895030e+11
...,...,...
86,2023-12-02,7.562494e+11
87,2023-12-03,7.718347e+11
88,2023-12-04,7.828452e+11
89,2023-12-05,8.205274e+11


In [4]:
merged_df

,timestamp,dominance,price
0,2023-09-08,46.97,26220.000000
1,2023-09-09,46.48,25907.228137
2,2023-09-10,46.54,25890.000000
3,2023-09-10,46.54,25835.000000
4,2023-09-12,46.81,25162.000000
...,...,...,...
85,2023-12-02,49.91,38690.000000
86,2023-12-03,49.76,39472.000000
87,2023-12-04,49.31,40002.000000
88,2023-12-05,50.77,41985.000000


In [5]:
# Convert 'timestamp' column to datetime in both DataFrames
merged_df['timestamp'] = pd.to_datetime(merged_df['timestamp'])
volume_df['timestamp'] = pd.to_datetime(volume_df['timestamp'])

# Merge the 'merged_df' and 'volume_df' DataFrames on the 'timestamp' column
final_df = pd.merge(merged_df, volume_df, on='timestamp', how='inner')

In [6]:
final_df

,timestamp,dominance,price,txn_volume
0,2023-09-08,46.97,26220.000000,5.098377e+11
1,2023-09-09,46.48,25907.228137,5.045331e+11
2,2023-09-10,46.54,25890.000000,5.043739e+11
3,2023-09-10,46.54,25835.000000,5.043739e+11
4,2023-09-12,46.81,25162.000000,4.895030e+11
...,...,...,...,...
85,2023-12-02,49.91,38690.000000,7.562494e+11
86,2023-12-03,49.76,39472.000000,7.718347e+11
87,2023-12-04,49.31,40002.000000,7.828452e+11
88,2023-12-05,50.77,41985.000000,8.205274e+11


In [7]:
# Calculate the 50-day moving average
final_df['moving_average_50d'] =final_df['price'].rolling(window=50).mean()

# Calculate the daily percentage change of the moving average
final_df['momentum_50d'] = final_df['moving_average_50d'].pct_change() * 100

# Applying 25% weight to dominance
final_df['weighted_dominance'] = final_df['dominance'] * 0.25

# Calculating the change in momentum percentage
final_df['momentum_change'] = final_df['momentum_50d'].diff()

# Applying 25% weightage (corrected weightage comment)
final_df['weighted_momentum_change'] = final_df['momentum_change'] * 0.25

# Calculating the combined score directly
final_df['combined_score'] = final_df['weighted_dominance'] + final_df['weighted_momentum_change']

In [8]:
final_df

,timestamp,dominance,price,txn_volume,moving_average_50d,momentum_50d,weighted_dominance,momentum_change,weighted_momentum_change,combined_score
0,2023-09-08,46.97,26220.000000,5.098377e+11,NaN,NaN,11.7425,NaN,NaN,NaN
1,2023-09-09,46.48,25907.228137,5.045331e+11,NaN,NaN,11.6200,NaN,NaN,NaN
2,2023-09-10,46.54,25890.000000,5.043739e+11,NaN,NaN,11.6350,NaN,NaN,NaN
3,2023-09-10,46.54,25835.000000,5.043739e+11,NaN,NaN,11.6350,NaN,NaN,NaN
4,2023-09-12,46.81,25162.000000,4.895030e+11,NaN,NaN,11.7025,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
85,2023-12-02,49.91,38690.000000,7.562494e+11,34562.788393,0.695195,12.4775,0.059083,0.014771,12.492271
86,2023-12-03,49.76,39472.000000,7.718347e+11,34815.028393,0.729802,12.4400,0.034607,0.008652,12.448652
87,2023-12-04,49.31,40002.000000,7.828452e+11,35077.859777,0.754937,12.3275,0.025134,0.006284,12.333784
88,2023-12-05,50.77,41985.000000,8.205274e+11,35374.559777,0.845833,12.6925,0.090896,0.022724,12.715224


In [9]:
# Calculate the daily percentage change in transaction volume
final_df['txn_volume_change'] = final_df['txn_volume'].pct_change() * 100

In [10]:
final_df

,timestamp,dominance,price,txn_volume,moving_average_50d,momentum_50d,weighted_dominance,momentum_change,weighted_momentum_change,combined_score,txn_volume_change
0,2023-09-08,46.97,26220.000000,5.098377e+11,NaN,NaN,11.7425,NaN,NaN,NaN,NaN
1,2023-09-09,46.48,25907.228137,5.045331e+11,NaN,NaN,11.6200,NaN,NaN,NaN,-1.040457
2,2023-09-10,46.54,25890.000000,5.043739e+11,NaN,NaN,11.6350,NaN,NaN,NaN,-0.031543
3,2023-09-10,46.54,25835.000000,5.043739e+11,NaN,NaN,11.6350,NaN,NaN,NaN,0.000000
4,2023-09-12,46.81,25162.000000,4.895030e+11,NaN,NaN,11.7025,NaN,NaN,NaN,-2.948401
...,...,...,...,...,...,...,...,...,...,...,...
85,2023-12-02,49.91,38690.000000,7.562494e+11,34562.788393,0.695195,12.4775,0.059083,0.014771,12.492271,2.520116
86,2023-12-03,49.76,39472.000000,7.718347e+11,34815.028393,0.729802,12.4400,0.034607,0.008652,12.448652,2.060867
87,2023-12-04,49.31,40002.000000,7.828452e+11,35077.859777,0.754937,12.3275,0.025134,0.006284,12.333784,1.426543
88,2023-12-05,50.77,41985.000000,8.205274e+11,35374.559777,0.845833,12.6925,0.090896,0.022724,12.715224,4.813486


In [11]:
#LOADING THE GOOGLE TRENDS
import pandas as pd

search_trends = pd.read_csv('multiTimeline-2.csv')  # Replace 'multiTimeline-2.csv' with the actual file name

# Assuming 'search_trends' has columns 'Time' and 'bitcoin: (India)'
# Convert 'Time' to datetime and 'bitcoin: (India)' to numeric
search_trends['Time'] = pd.to_datetime(search_trends['Time'])
search_trends['bitcoin: (India)'] = pd.to_numeric(search_trends['bitcoin: (India)'], errors='coerce')

# Take the average of 'bitcoin: (India)' column
trends_avg = search_trends['bitcoin: (India)'].mean()


In [14]:
# Calculate the combined score directly in final_df
final_df['combined_score'] = final_df['weighted_dominance'] + final_df['weighted_momentum_change']

# Calculate the daily percentage change in transaction volume
final_df['txn_volume_change'] = final_df['txn_volume'].pct_change() * 100

# Function to return the final score for a given date
def final_score(date, volatility, df, google_trends_value):
    # Filter the DataFrame for the specified date
    selected_row = df[df['timestamp'] == date]

    # Check if the date exists in the DataFrame
    if selected_row.empty:
        return None

    # Extract the combined score from the selected row
    combined_score = selected_row['combined_score'].values[0]

    # Add volatility with 25% weightage
    combined_score += volatility * 0.25

    # Add Google Trends value with 25% weightage
    combined_score += google_trends_value * 0.25

    return combined_score



In [16]:
final_df

,timestamp,dominance,price,txn_volume,moving_average_50d,momentum_50d,weighted_dominance,momentum_change,weighted_momentum_change,combined_score,txn_volume_change
0,2023-09-08,46.97,26220.000000,5.098377e+11,NaN,NaN,11.7425,NaN,NaN,NaN,NaN
1,2023-09-09,46.48,25907.228137,5.045331e+11,NaN,NaN,11.6200,NaN,NaN,NaN,-1.040457
2,2023-09-10,46.54,25890.000000,5.043739e+11,NaN,NaN,11.6350,NaN,NaN,NaN,-0.031543
3,2023-09-10,46.54,25835.000000,5.043739e+11,NaN,NaN,11.6350,NaN,NaN,NaN,0.000000
4,2023-09-12,46.81,25162.000000,4.895030e+11,NaN,NaN,11.7025,NaN,NaN,NaN,-2.948401
...,...,...,...,...,...,...,...,...,...,...,...
85,2023-12-02,49.91,38690.000000,7.562494e+11,34562.788393,0.695195,12.4775,0.059083,0.014771,12.492271,2.520116
86,2023-12-03,49.76,39472.000000,7.718347e+11,34815.028393,0.729802,12.4400,0.034607,0.008652,12.448652,2.060867
87,2023-12-04,49.31,40002.000000,7.828452e+11,35077.859777,0.754937,12.3275,0.025134,0.006284,12.333784,1.426543
88,2023-12-05,50.77,41985.000000,8.205274e+11,35374.559777,0.845833,12.6925,0.090896,0.022724,12.715224,4.813486


In [15]:
# Example:
input_date = '2023-12-06'
volatility = 61  

# Use the average Google Trends value as a parameter
google_trends_value = trends_avg

result = final_score(input_date, volatility, final_df, google_trends_value)

if result is not None:
    print(f"The final score for {input_date} is: {result}")
else:
    print(f"No data available for {input_date}")


The final score for 2023-12-06 is: 38.73351629993881
